In [46]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data_EDA.csv')

In [3]:
df.head()

ABS ActiveSafetySysNote AdaptiveCruiseControl AdaptiveDrivingBeam  \
0       NaN                 NaN                   NaN                 NaN   
1       NaN                 NaN                   NaN                 NaN   
2  Standard                 NaN              Optional                 NaN   
3  Standard                 NaN                   NaN                 NaN   
4       NaN                 NaN                   NaN                 NaN   

  AirBagLocCurtain                  AirBagLocFront  \
0         All Rows  1st Row (Driver and Passenger)   
1              NaN  1st Row (Driver and Passenger)   
2         All Rows  1st Row (Driver and Passenger)   
3         All Rows  1st Row (Driver and Passenger)   
4              NaN  1st Row (Driver and Passenger)   

                    AirBagLocKnee AirBagLocSeatCushion  \
0                             NaN                  NaN   
1                             NaN                  NaN   
2                             NaN                  NaN   
3  1st Row (Driver and Passenger)                  NaN   
4                             NaN                  NaN   

                    AirBagLocSide AutoReverseSystem  ...    type paint_color  \
0  1st Row (Driver and Passenger)               NaN  ...  pickup       white   
1  1st Row (Driver and Passenger)               NaN  ...  pickup        blue   
2  1st Row (Driver and Passenger)          Standard  ...  pickup         red   
3  1st Row (Driver and Passenger)          Standard  ...  pickup         red   
4                             NaN               NaN  ...  pickup       black   

                                           image_url  \
0  https://images.craigslist.org/00R0R_lwWjXSEWNa...   
1  https://images.craigslist.org/00R0R_lwWjXSEWNa...   
2  https://images.craigslist.org/01212_jjirIWa0y0...   
3  https://images.craigslist.org/00x0x_1y9kIOzGCF...   
4  https://images.craigslist.org/00V0V_ftJbKrOgdi...   

                                         description county state    lat  \
0  Carvana is the safer way to buy a car During t...    NaN    al  32.59   
1  Carvana is the safer way to buy a car During t...    NaN    al  32.59   
2  Carvana is the safer way to buy a car During t...    NaN    al  32.59   
3  Carvana is the safer way to buy a car During t...    NaN    al  32.59   
4  Carvana is the safer way to buy a car During t...    NaN    al  32.59   

    long               posting_date  Year  
0 -85.48  2021-05-04 17:31:18+00:00  2014  
1 -85.48  2021-05-04 17:31:08+00:00  2010  
2 -85.48  2021-05-04 17:31:25+00:00  2020  
3 -85.48  2021-05-04 15:41:31+00:00  2017  
4 -85.48  2021-05-03 18:41:25+00:00  2012  

[5 rows x 110 columns]

In [4]:
df.shape

(90493, 110)

In [5]:
cats = ['GVWR', 'ABS', 'AirBagLocCurtain', 'AirBagLocSide', 'BodyClass', 'BrakeSystemType','Doors', 'DriveType', 
        'DynamicBrakeSupport', 'ESC', 'EngineConfiguration', 'EngineCylinders', 'FuelInjectionType','FuelTypePrimary',
        'KeylessIgnition', 'RearVisibilitySystem', 'ValveTrainDesign', 'Turbo', 'ValveTrainDesign']

extra_cats = ['AdaptiveCruiseControl', 'AdaptiveDrivingBeam', 'AirBagLocKnee', 'AutoReverseSystem','BlindSpotMon',
            'CAN_AACN', 'CIB', 'EngineCycles', 'ForwardCollisionWarning', 'FuelTypeSecondary',
            'LaneDepartureWarning', 'LaneKeepSystem', 'LowerBeamHeadlampLightSource',  'PedestrianAutomaticEmergencyBraking',
            'Pretensioner', 'SeatRows', 'Seats', 'SemiautomaticHeadlampBeamSwitching', 'TractionControl', 'TransmissionSpeeds',
            'TransmissionStyle', 'WheelSizeFront', 'WheelSizeRear', 'Windows']

cl_cats = ['condition', 'transmission', 'drive', 'size', 'type', 'paint_color']

veh_indicators = ['Make', 'Model', 'Series','Series2','Trim','Trim2',
    'VehicleDescriptor', 'VehicleType', 'VIN']

target=['price']

In [6]:
df_cat = df[cats + extra_cats + cl_cats]
df_unscaled = df[['BasePrice', 'DisplacementCC', 'EngineHP', 'Year', 'odometer']]

In [7]:
df_dummies = pd.get_dummies(df_cat.astype(str))

In [22]:
new_df = pd.concat([df_unscaled, df_dummies, df['price']], axis=1)
new_df.shape

(90493, 301)

In [53]:
from sklearn.model_selection import train_test_split

X = new_df.drop(['price'], axis=1)
y = new_df[['price']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [54]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(72394, 300)
(72394, 1)
(18099, 300)
(18099, 1)


In [55]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [63]:
indices = []

for k,v in y.value_counts().to_dict().items():
    if v == 1:
        indices.append(y[y['price'] == k].index[0])
print(len(indices))

6323


In [57]:
from sklearn.ensemble import HistGradientBoostingClassifier

# Initialize HistGradientBoostingClassifier
clf = HistGradientBoostingClassifier(max_iter=100, random_state=42)

# Train the classifier
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

C:\Users\mitch\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [14]:
from sklearn import datasets
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

In [16]:
# k = 4 tells four top features to be selected
# Score function Chi2 tells the feature to be selected using Chi Square
test = SelectKBest(score_func=chi2, k=4)
fit = test.fit(X, y)
fit.scores_

ValueError: Input X contains NaN.
SelectKBest does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [11]:
from sklearn.metrics import mean_squared_error, r2_score 
from sklearn import linear_model, preprocessing


# Create the model: make a variable called rModel, and use it linear_model.LinearRegression appropriately
rModel = linear_model.LinearRegression(normalize=True)
rModel.fit(X_train, y_train)
# Evaluate the model  
print(rModel.score(X_train, y_train))

C:\Users\mitch\anaconda3\Lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(


ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [13]:
from sklearn import tree, metrics

# Declare a variable called entr_model and use tree.DecisionTreeClassifier. 
entr_model = tree.DecisionTreeClassifier(criterion='entropy', random_state = 1234)

# Call fit() on entr_model
entr_model.fit(X_train, y_train) 

# Call predict() on entr_model with X_test passed to it, and assign the result to a variable y_pred 
y_pred = entr_model.predict(X_test) 

# Call Series on our y_pred variable with the following: pd.Series(y_pred)
pd.Series(y_pred)

# Check out entr_model
entr_model

ValueError: Input X contains NaN.
DecisionTreeClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values